In [1]:
import os
import wave
import time
import pickle
import pyaudio
import warnings
import numpy as np
from sklearn import preprocessing
from scipy.io.wavfile import read
import python_speech_features as mfcc
from sklearn.mixture import GaussianMixture 

In [2]:
def record_audio_train():
	Name =(input("Please Enter Your Name:"))
	for count in range(5):
		FORMAT = pyaudio.paInt16
		CHANNELS = 1
		RATE = 44100
		CHUNK = 512
		RECORD_SECONDS = 10
		device_index = 2
		audio = pyaudio.PyAudio()
		print("----------------------record device list---------------------")
		info = audio.get_host_api_info_by_index(0)
		numdevices = info.get('deviceCount')
		for i in range(0, numdevices):
		        if (audio.get_device_info_by_host_api_device_index(0, i).get('maxInputChannels')) > 0:
		            print("Input Device id ", i, " - ", audio.get_device_info_by_host_api_device_index(0, i).get('name'))
		print("-------------------------------------------------------------")
		index = int(input())		
		print("recording via index "+str(index))
		stream = audio.open(format=FORMAT, channels=CHANNELS,
		                rate=RATE, input=True,input_device_index = index,
		                frames_per_buffer=CHUNK)
		print ("recording started")
		Recordframes = []
		for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
		    data = stream.read(CHUNK)
		    Recordframes.append(data)
		print ("recording stopped")
		stream.stop_stream()
		stream.close()
		audio.terminate()
		OUTPUT_FILENAME=Name+"-sample"+str(count)+".wav"
		WAVE_OUTPUT_FILENAME=os.path.join("training_set",OUTPUT_FILENAME)
		trainedfilelist = open("training_set_addition.txt", 'a')
		trainedfilelist.write(OUTPUT_FILENAME+"\n")
		waveFile = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
		waveFile.setnchannels(CHANNELS)
		waveFile.setsampwidth(audio.get_sample_size(FORMAT))
		waveFile.setframerate(RATE)
		waveFile.writeframes(b''.join(Recordframes))
		waveFile.close()

In [3]:
record_audio_train()

Please Enter Your Name: Sipa


----------------------record device list---------------------
Input Device id  0  -  iPhone Microphone
Input Device id  1  -  MacBook Air Microphone
-------------------------------------------------------------


 1


recording via index 1
recording started
recording stopped
----------------------record device list---------------------
Input Device id  0  -  iPhone Microphone
Input Device id  1  -  MacBook Air Microphone
-------------------------------------------------------------


 1


...

recording via index 1
recording started
recording stopped
----------------------record device list---------------------
Input Device id  0  -  iPhone Microphone
Input Device id  1  -  MacBook Air Microphone
-------------------------------------------------------------


 1


recording via index 1
recording started
recording stopped
----------------------record device list---------------------
Input Device id  0  -  iPhone Microphone
Input Device id  1  -  MacBook Air Microphone
-------------------------------------------------------------


 1


recording via index 1
recording started
recording stopped


In [4]:
def calculate_delta(array):
   
    rows,cols = array.shape
    print(rows)
    print(cols)
    deltas = np.zeros((rows,20))
    N = 2
    for i in range(rows):
        index = []
        j = 1
        while j <= N:
            if i-j < 0:
              first =0
            else:
              first = i-j
            if i+j > rows-1:
                second = rows-1
            else:
                second = i+j 
            index.append((second,first))
            j+=1
        deltas[i] = ( array[index[0][0]]-array[index[0][1]] + (2 * (array[index[1][0]]-array[index[1][1]])) ) / 10
    return deltas

In [5]:
def extract_features(audio,rate):
       
    mfcc_feature = mfcc.mfcc(audio,rate, 0.025, 0.01,20,nfft = 1200, appendEnergy = True)    
    mfcc_feature = preprocessing.scale(mfcc_feature)
    print(mfcc_feature)
    delta = calculate_delta(mfcc_feature)
    combined = np.hstack((mfcc_feature,delta)) 
    return combined

In [6]:
def train_model():

	source   = "/Users/priyabratkumarbishwal/Documents/Techie/Python/SCBAccelerate/training_set/"   
	dest = "/Users/priyabratkumarbishwal/Documents/Techie/Python/SCBAccelerate/trained_models/"
	train_file = "/Users/priyabratkumarbishwal/Documents/Techie/Python/SCBAccelerate/training_set_addition.txt"        
	file_paths = open(train_file,'r')
	count = 1
	features = np.asarray(())
	for path in file_paths:    
	    path = path.strip()   
	    print(path)

	    sr,audio = read(source + path)
	    print(sr)
	    vector   = extract_features(audio,sr)
	    
	    if features.size == 0:
	        features = vector
	    else:
	        features = np.vstack((features, vector))

	    if count == 5:    
	        gmm = GaussianMixture(n_components = 6, max_iter = 200, covariance_type='diag',n_init = 3)
	        gmm.fit(features)
	        
	        # dumping the trained gaussian model
	        picklefile = path.split("-")[0]+".gmm"
	        pickle.dump(gmm,open(dest + picklefile,'wb'))
	        print('+ modeling completed for speaker:',picklefile," with data point = ",features.shape)   
	        features = np.asarray(())
	        count = 0
	    count = count + 1

In [11]:
#record_audio_train()

In [7]:
train_model()

Priyansh-sample0.wav
44100
[[ 0.27252405 -2.03240848 -0.70247871 ... -0.38403691 -0.41375112
  -0.42167733]
 [-0.77080081 -1.59094896 -0.90116633 ... -0.64425489 -1.23474692
   0.37444885]
 [-2.63832561 -3.53059396  1.70238887 ...  0.08704399 -0.15623069
   0.19783276]
 ...
 [ 0.59955301  0.82918218  0.95265061 ... -0.41484449  0.41236833
  -1.66156727]
 [ 0.57337776  0.85380752  0.83651504 ... -0.5753309   0.28093051
  -1.03487406]
 [ 0.39526209  0.88310135  0.81172593 ... -0.09498072  0.01155935
  -1.70212065]]
999
20
Priyansh-sample1.wav
44100
[[-0.41872877  0.52501222  1.27990446 ... -0.25137651 -0.65050839
  -0.93515255]
 [-2.39545513 -0.81015643  0.79010915 ...  0.0469533  -1.43909096
   0.31388676]
 [-4.20034601 -3.65676021  2.15883991 ... -0.21530978 -0.71690714
   0.21929557]
 ...
 [-2.48918559 -0.11065332  1.12984932 ...  0.52717947  0.30178685
   0.49274299]
 [-2.46880586 -0.12219681  1.10737183 ... -0.27794888 -0.09790758
   0.55071312]
 [-2.76196998 -0.27029692  0.8874822 

In [9]:
def record_audio_test():

	FORMAT = pyaudio.paInt16
	CHANNELS = 1
	RATE = 44100
	CHUNK = 512
	RECORD_SECONDS = 10
	device_index = 2
	audio = pyaudio.PyAudio()
	print("----------------------record device list---------------------")
	info = audio.get_host_api_info_by_index(0)
	numdevices = info.get('deviceCount')
	for i in range(0, numdevices):
	        if (audio.get_device_info_by_host_api_device_index(0, i).get('maxInputChannels')) > 0:
	            print("Input Device id ", i, " - ", audio.get_device_info_by_host_api_device_index(0, i).get('name'))
	print("-------------------------------------------------------------")
	index = int(input())		
	print("recording via index "+str(index))
	stream = audio.open(format=FORMAT, channels=CHANNELS,
	                rate=RATE, input=True,input_device_index = index,
	                frames_per_buffer=CHUNK)
	print ("recording started")
	Recordframes = []
	for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
	    data = stream.read(CHUNK)
	    Recordframes.append(data)
	print ("recording stopped")
	stream.stop_stream()
	stream.close()
	audio.terminate()
	OUTPUT_FILENAME="sample.wav"
	WAVE_OUTPUT_FILENAME=os.path.join("testing_set",OUTPUT_FILENAME)
	trainedfilelist = open("testing_set_addition.txt", 'a')
	trainedfilelist.write(OUTPUT_FILENAME+"\n")
	waveFile = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
	waveFile.setnchannels(CHANNELS)
	waveFile.setsampwidth(audio.get_sample_size(FORMAT))
	waveFile.setframerate(RATE)
	waveFile.writeframes(b''.join(Recordframes))
	waveFile.close()

In [ ]:
record_audio_test()

----------------------record device list---------------------
Input Device id  0  -  iPhone Microphone
Input Device id  1  -  MacBook Air Microphone
-------------------------------------------------------------


 1


recording via index 1
recording started


In [15]:
def test_model():

	source   = "/Users/priyabratkumarbishwal/Documents/Techie/Python/SCBAccelerate/testing_set/"  
	modelpath = "/Users/priyabratkumarbishwal/Documents/Techie/Python/SCBAccelerate/trained_models/"
	test_file = "/Users/priyabratkumarbishwal/Documents/Techie/Python/SCBAccelerate/testing_set_addition.txt"       
	file_paths = open(test_file,'r')
	 
	gmm_files = [os.path.join(modelpath,fname) for fname in
	              os.listdir(modelpath) if fname.endswith('.gmm')]
	 
	#Load the Gaussian gender Models
	models    = [pickle.load(open(fname,'rb')) for fname in gmm_files]
	speakers   = [fname.split("\\")[-1].split(".gmm")[0] for fname 
	              in gmm_files]
	 
	# Read the test directory and get the list of test audio files 
	for path in file_paths:   
	     
	    path = path.strip()   
	    print(path)
	    sr,audio = read(source + path)
	    vector   = extract_features(audio,sr)
	     
	    log_likelihood = np.zeros(len(models)) 
	    
	    for i in range(len(models)):
	        gmm    = models[i]  #checking with each model one by one
	        scores = np.array(gmm.score(vector))
	        log_likelihood[i] = scores.sum()
	     
	    winner = np.argmax(log_likelihood)
	    print("\tdetected as - ", speakers[winner])
	    time.sleep(1.0)  

In [18]:
test_model()

sample.wav
[[-0.08129313  0.05725066  1.188978   ...  0.2074799  -0.61949529
  -0.51959785]
 [-0.6904427  -1.30384154 -0.67571386 ...  0.52663831 -0.609395
  -0.36954817]
 [-2.70529277 -2.78519029  1.85634806 ... -0.1449448  -0.10625497
   0.0939083 ]
 ...
 [ 0.25284846 -1.81461326 -1.89981261 ...  0.08893606  0.02975997
  -0.10701443]
 [ 0.12511353 -1.33917673 -1.46364711 ...  0.13854701 -0.45306907
  -0.0743472 ]
 [-0.05565771 -1.21491051 -1.4005626  ...  0.03461598 -0.44501382
  -0.05094169]]
999
20
	detected as -  /Users/priyabratkumarbishwal/Documents/Techie/Python/SCBAccelerate/trained_models/Priyansh
